## imports

In [74]:
import pandas as pd
import numpy  as np
import json

from geopy.distance import geodesic

## readin locations data

In [75]:
with open('../Data/rows.geojson') as f:
    data = json.load(f)

In [76]:
properties = pd.DataFrame()

for i in range(0, len(data['features'])):
    properties = properties.append(pd.DataFrame(data['features'][i]['properties'], index = [i]))

properties = properties.loc[: , ["plot_size",
                                 "centroid_y",
                                 "centroid_x",
                                 "fractal_order",
                                 "location"]]

properties.rename(columns={'centroid_y':'longitude', 'centroid_x':'latitude'}, inplace=True)
properties["longlat"] = properties.apply(lambda x: [x.longitude, x.latitude], axis=1)
SO = properties.loc[properties.fractal_order == 2, : ]

In [77]:
def closest(pt, others):
    
    clst_pt = min(others.longlat, key = lambda x: geodesic(pt, x).meters)
        
    return others.location.loc[others.longlat.apply(lambda x: x == clst_pt)].reset_index(drop = True)

## readin data

In [78]:
trees_df = pd.read_csv("../Results/trees_sorted.csv")
mamls_df = pd.read_csv("../Results/mammals_sorted.csv")
btles_df = pd.read_csv("../Results/beetles_sorted.csv")

fpn = pd.read_csv("../Data/Fractal_point_nesting.csv")
agb = pd.read_csv("../Data/AGB.csv")

In [79]:
fpn["FirstOrder"] = fpn.FirstOrder.str.partition("_")[2].astype(int)

fractals = fpn.loc[:, ["FirstOrder", "SecondOrder"]]

## sort beetles (match to second order fragment)

In [80]:
btles_df = btles_df.merge(fractals, how = "left", left_on = "trap_N", right_on = "FirstOrder")
btles_df["SecondOrder"] = btles_df.SecondOrder.str.partition("_")[2].astype(int)
btles_df = btles_df.merge(agb[["Plot", "AGB_Chave_moist", "ForestQuality"]], how = "left",
                          left_on = "SecondOrder", right_on = "Plot")

In [120]:
btles_agb = pd.DataFrame(btles_df.groupby("block").AGB_Chave_moist.median())

btles_agb.to_csv("../Results/btles_agb.csv")

## sort mammals

In [81]:
mamls_df.Trap_ID = mamls_df.Trap_ID.apply(lambda x: x[:-1])
mamls_df = mamls_df.merge(properties[["location", "longlat"]],
                          how = "left", left_on = "Trap_ID", right_on = "location")

In [82]:
mamls_df["SecondOrder"] = mamls_df.longlat.apply(lambda x: closest(x, SO))
mamls_df.SecondOrder = mamls_df.SecondOrder.str.partition("_")[2].astype(int)

In [83]:
mamls_df = mamls_df.merge(agb[["Plot", "AGB_Chave_moist", "ForestQuality"]], how = "left",
                          left_on = "SecondOrder", right_on = "Plot")

In [128]:
mamls_df["plot"] = mamls_df.Trap_ID.str.rpartition("-")[0]

In [160]:
mamls_agb = pd.DataFrame(mamls_df.groupby("plot").AGB_Chave_moist.median())
mamls_agb.to_csv("../Results/mamls_agb.csv")

## sort trees

In [103]:
trees_agb = pd.DataFrame(trees_df.groupby(["plot", "census"]).stem_C.sum())

In [104]:
trees_agb = pd.DataFrame(trees_agb.groupby("plot").mean()*  0.0625 * 0.001)

In [114]:
trees_agb.to_csv("../Results/trees_agb.csv")